In [7]:
import requests
import pandas as pd
from io import StringIO

# API URL ve Client ID
url = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/statistic/export?id=oy5zjitsmn&fileformat=CSV"
headers = {"X-IBM-Client-Id": "f943a426a663183091a1401f55eec60d"}

# API'den veriyi çek
response = requests.get(url, headers=headers)

In [8]:
if response.status_code == 200:
    # Veriyi StringIO ile oku
    data = StringIO(response.content.decode('latin1'))
    df_housing = pd.read_csv(data, sep=',', encoding='latin1')  # SEP = ','
else:
    print(f"Veri çekilemedi! Status code: {response.status_code}")

In [9]:
df_housing.head()

,Dim-00:TEMPS,Dim-01:TERRITORI,Dim-01:TERRITORI (order),Dim-01:TERRITORI (type),Dim-02:ÃS I DESTINACIÃ PRINCIPAL,VALUE
0,2009-01-01T00:00:00Z,el Coll,29,Barri,Ensenyament i cultura,29.0
1,2009-01-01T00:00:00Z,el Carmel,37,Barri,Oficines,77.0
2,2009-01-01T00:00:00Z,la Marina de Port,13,Barri,Altres,12.0
3,2009-01-01T00:00:00Z,el Parc i la Llacuna del Poblenou,66,Barri,Habitatge,6434.0
4,2009-01-01T00:00:00Z,la Font de la Guatlla,14,Barri,Esportiu,6.0


In [10]:
df_housing.rename(columns={
    'Dim-00:TEMPS': 'year',
    'Dim-01:TERRITORI': 'territory',
    'Dim-01:TERRITORI (order)': 'territory_order',
    'Dim-01:TERRITORI (type)': 'territory_type',
    'Dim-02:ÃS I DESTINACIÃ PRINCIPAL': 'main_use_destination',
    'VALUE': 'num_premises'
}, inplace=True)


In [13]:
df_housing['territory_type'].unique()

array(['Barri', 'Districte', 'Municipi'], dtype=object)

In [14]:
df_housing['territory'] = df_housing['territory'].str.encode('latin1').str.decode('utf-8')
df_housing['territory_type'] = df_housing['territory_type'].str.encode('latin1').str.decode('utf-8')
df_housing['main_use_destination'] = df_housing['main_use_destination'].str.encode('latin1').str.decode('utf-8')

In [15]:
df_housing['main_use_destination'].unique()

array(['Ensenyament i cultura', 'Oficines', 'Altres', 'Habitatge',
       'Esportiu', 'Turisme i hostaleria', 'Religiós', 'Comerç',
       'Espectacles', 'Aparcaments', 'Indústria', 'Sanitat', 'Magatzem',
       'Administració Pública', 'No consta', 'Sòl sense edificar'],
      dtype=object)

In [16]:
df_housing['main_use_destination'] = df_housing['main_use_destination'].replace({
    'Ensenyament i cultura': 'Education and culture',
    'Oficines': 'Offices',
    'Altres': 'Other',
    'Habitatge': 'Housing',
    'Esportiu': 'Sports',
    'Turisme i hostaleria': 'Tourism and hospitality',
    'Religiós': 'Religious',
    'Comerç': 'Commerce',
    'Espectacles': 'Entertainment',
    'Aparcaments': 'Parking',
    'Indústria': 'Industry',
    'Sanitat': 'Healthcare',
    'Magatzem': 'Warehouse',
    'Administració Pública': 'Public administration',
    'No consta': 'Not specified',
    'Sòl sense edificar': 'Unbuilt land'
})


In [17]:
df_housing['main_use_destination'].unique()

array(['Education and culture', 'Offices', 'Other', 'Housing', 'Sports',
       'Tourism and hospitality', 'Religious', 'Commerce',
       'Entertainment', 'Parking', 'Industry', 'Healthcare', 'Warehouse',
       'Public administration', 'Not specified', 'Unbuilt land'],
      dtype=object)

In [18]:
df_housing['year'].unique()

array(['2009-01-01T00:00:00Z', '2010-01-01T00:00:00Z',
       '2011-01-01T00:00:00Z', '2012-01-01T00:00:00Z',
       '2013-01-01T00:00:00Z', '2014-01-01T00:00:00Z',
       '2015-01-01T00:00:00Z', '2016-01-01T00:00:00Z',
       '2017-01-01T00:00:00Z', '2018-01-01T00:00:00Z',
       '2019-01-01T00:00:00Z', '2020-01-01T00:00:00Z',
       '2021-01-01T00:00:00Z', '2022-01-01T00:00:00Z',
       '2023-03-06T00:00:00Z', '2024-02-26T00:00:00Z',
       '2025-03-04T00:00:00Z'], dtype=object)

In [19]:
df_housing['year'] = pd.to_datetime(df_housing['year'], errors='coerce').dt.year


In [20]:
print(sorted(df_housing['year'].unique()))


[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


In [21]:
df_housing['territory_type'] = df_housing['territory_type'].replace({
    'Barri': 'Neighbourhood',
    'Districte': 'District',
    'Municipi': 'Municipality'
})


In [22]:
df_housing['territory_type'].unique()

array(['Neighbourhood', 'District', 'Municipality'], dtype=object)

In [23]:
df_housing.head()

,year,territory,territory_order,territory_type,main_use_destination,num_premises
0,2009,el Coll,29,Neighbourhood,Education and culture,29.0
1,2009,el Carmel,37,Neighbourhood,Offices,77.0
2,2009,la Marina de Port,13,Neighbourhood,Other,12.0
3,2009,el Parc i la Llacuna del Poblenou,66,Neighbourhood,Housing,6434.0
4,2009,la Font de la Guatlla,14,Neighbourhood,Sports,6.0


In [36]:
df_housing.to_csv("housing_stock_data.csv", index=False, encoding='utf-8')


## VACANT DATA 

In [24]:
import requests
import pandas as pd
from io import StringIO

# 1. API bilgileri
url_conservation = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/statistic/export?id=rdkoh4ukx5&fileformat=CSV"
headers = {"X-IBM-Client-Id": "f943a426a663183091a1401f55eec60d"}

# 2. Veri çek
response_conservation = requests.get(url_conservation, headers=headers)

# 3. Status kontrol
if response_conservation.status_code == 200:
    data_conservation = StringIO(response_conservation.content.decode('latin1'))
    df_conservation = pd.read_csv(data_conservation, sep=',')

    print("✅ Dataset çekildi, ilk 5 satır:")
    print(df_conservation.head())
    print("\n📌 Kolonlar:")
    print(df_conservation.columns.tolist())

else:
    print(f"❌ HATA: Status code {response_conservation.status_code}")


✅ Dataset çekildi, ilk 5 satır:
           Dim-00:TEMPS Dim-01:TERRITORI  Dim-01:TERRITORI (order)  \
0  1981-01-01T00:00:00Z        Barcelona                         1   
1  1981-01-01T00:00:00Z        Barcelona                         1   
2  1981-01-01T00:00:00Z        Barcelona                         1   
3  1981-01-01T00:00:00Z        Barcelona                         1   
4  1991-01-01T00:00:00Z  Horta-GuinardÃ³                         7   

  Dim-01:TERRITORI (type) Dim-02:TIPUS HABITATGE     VALUE  
0                Municipi              Secundari   14399.0  
1                Municipi                 Altres       0.0  
2                Municipi              Principal  579831.0  
3                Municipi              Desocupat   79276.0  
4               Districte              Secundari    1899.0  

📌 Kolonlar:
['Dim-00:TEMPS', 'Dim-01:TERRITORI', 'Dim-01:TERRITORI (order)', 'Dim-01:TERRITORI (type)', 'Dim-02:TIPUS HABITATGE', 'VALUE']


In [25]:
df_conservation.rename(columns={
    'Dim-00:TEMPS': 'year',
    'Dim-01:TERRITORI': 'territory',
    'Dim-01:TERRITORI (order)': 'territory_order',
    'Dim-01:TERRITORI (type)': 'territory_type',
    'Dim-02:TIPUS HABITATGE': 'occupancy_status',
    'VALUE': 'num_housing_units'
}, inplace=True)


In [31]:
df_conservation['year'].unique()

array(['1981-01-01T00:00:00Z', '1991-01-01T00:00:00Z',
       '2001-01-01T00:00:00Z', '2011-01-01T00:00:00Z',
       '2021-01-01T00:00:00Z'], dtype=object)

In [28]:
df_conservation['occupancy_status'] = df_conservation['occupancy_status'].replace({
    'Principal': 'Main',
    'Secundari': 'Secondary',
    'Desocupat': 'Vacant',
    'Altres': 'Other',
    'Allotjament': 'Accommodation',
    'No principal': 'Not main'
})


In [29]:
df_conservation['territory_type'] = df_conservation['territory_type'].replace({
    'Municipi': 'Municipality',
    'Districte': 'District',
    'Barri': 'Neighbourhood'
})


In [30]:
df_conservation['territory'] = df_conservation['territory'].str.encode('latin1').str.decode('utf-8')
df_conservation['territory_type'] = df_conservation['territory_type'].str.encode('latin1').str.decode('utf-8')

In [32]:
df_conservation['year'] = pd.to_datetime(df_conservation['year'], errors='coerce').dt.year

In [33]:
df_conservation.head()

,year,territory,territory_order,territory_type,occupancy_status,num_housing_units
0,1981,Barcelona,1,Municipality,Secondary,14399.0
1,1981,Barcelona,1,Municipality,Other,0.0
2,1981,Barcelona,1,Municipality,Main,579831.0
3,1981,Barcelona,1,Municipality,Vacant,79276.0
4,1991,Horta-Guinardó,7,District,Secondary,1899.0


In [35]:
df_conservation.to_csv("housing_occupancy_data.csv", index=False, encoding='utf-8')
